In [ ]:
import numpy as np
import itertools
import math
import scipy
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib import transforms
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import dask
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import netCDF4
import matplotlib.image as imag
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA,IncrementalPCA
from yellowbrick.cluster import KElbowVisualizer
from matplotlib import ticker
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from mpl_toolkits.basemap import Basemap
from statistics import mode
from scipy.ndimage.filters import uniform_filter1d
from sklearn.utils import shuffle
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

In [ ]:
def plot_generator_paper(sample, X, Z):
    
    fz = 15*1.25
    lw = 4
    siz = 100
    XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
    XTEXT = 0.25 # Text placement
    YTEXT = 0.3 # Text placement
    
    plt.rc('text', usetex=False)
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    #mpl.rcParams["font.serif"] = "STIX"
    plt.rc('font', family='serif', size=fz)
    matplotlib.rcParams['lines.linewidth'] = lw
    
    
    cmap="RdBu_r"
    fig, ax = plt.subplots(1,1, figsize=(15,6))
    cs0 = ax.pcolor(X, Z, sample, cmap=cmap, vmin=-1.0, vmax = 1.0)
    ax.set_title("Anomalous Vertical Velocity Field Detected By ELBO")
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_xlabel("CRMs", fontsize=fz*1.5)
    ax.xaxis.set_label_coords(0.54,-0.05)
    h = ax.set_ylabel("hPa", fontsize = fz*1.5)
    h.set_rotation(0)
    ax.yaxis.set_label_coords(-0.10,0.44)
    #y_ticks = np.arange(1350, 0, -350)
    #ax.set_yticklabels(y_ticks, fontsize=fz*1.33)
    ax.tick_params(axis='x', labelsize=fz*1.33)
    ax.tick_params(axis='y', labelsize=fz*1.33)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(cs0, cax=cax)
    cbar.set_label(label=r'$\left(\mathrm{m\ s^{-1}}\right)$', rotation="horizontal", fontsize=fz*1.5, labelpad=30, y = 0.65)
    plt.show()
    #plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Anomaly.pdf")
    
#plot_generator(test[0,:,:])

In [ ]:
fz = 15*1.5
lw = 4
siz = 100
XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
XTEXT = 0.25 # Text placement
YTEXT = 0.3 # Text placement

plt.rc('text', usetex=False)
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
#mpl.rcParams["font.serif"] = "STIX"
plt.rc('font', family='serif', size=fz)
matplotlib.rcParams['lines.linewidth'] = lw

In [ ]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-00000.nc")
levs = np.array(others.variables['lev'])
lons = np.array(others.variables['lon'])
new = np.flip(levs)
crms = np.arange(1,129,1)
Xs, Zs = np.meshgrid(crms, new)

In [ ]:
Diurnal_Amazon_W_Test_2D = np.load("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/Synoptic_Latent_Spaces/Single_Day_2D_PCA_Latent_Space__369.npy")
Diurnal_Atlantic_W_Test_2D = np.load("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/Synoptic_Latent_Spaces/Atlantic_Day_2D_PCA_Latent_Space__369.npy")

z_test_tsne = np.load("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/Synoptic_Latent_Spaces/Full_Trackable_2D_PCA_Latent_Space__369.npy")


Test_Images = np.load("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_day.npy")
Ocean_Test_Images = np.load("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_atlantic_test_day.npy")
All_Test_Images = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_W_Test.npy")
Max_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Max_Scalar.npy")
Min_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Min_Scalar.npy")
Test_Images = np.interp(Test_Images, (0, 1), (Min_Scalar, Max_Scalar))
All_Test_Images = np.interp(All_Test_Images, (0, 1), (Min_Scalar, Max_Scalar))

In [ ]:
colors = cm.hsv(np.linspace(0, 1, int(len(Diurnal_Amazon_W_Test_2D)/4)))
bc_labels = ["0","1","2","3", "4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
fig, ax = plt.subplots(figsize=(30, 10))
ax.scatter(x=z_test_tsne[:, 0], y=z_test_tsne[:, 1], c="black", s=0.2)
count = -1
for i in range(len(Diurnal_Amazon_W_Test_2D)):
    if i%4 == 0:
        count = count+1
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400, label=str(int(i/4)))
    else:
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400)
        
#ax.legend(bbox_to_anchor=(0.999, 1.00))
ax.legend(loc="best", ncol=4, fontsize=fz*0.75)
#ax.legend(loc="lower left", mode = "expand", ncol=24, fontsize=fz/2)
ax.set_title("Latent Space from Amazon Average Diurnal Cycle Colored by LST Hour", fontsize=fz*2)

#plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Amazon.pdf")
print(" ")

In [ ]:
colors = cm.hsv(np.linspace(0, 1, int(len(Diurnal_Amazon_W_Test_2D)/4)))
bc_labels = ["0","1","2","3", "4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
fig, ax = plt.subplots(figsize=(30, 10))
ax.scatter(x=z_test_tsne[:, 0], y=z_test_tsne[:, 1], c="black", s=0.2)
count = -1
for i in range(len(Diurnal_Amazon_W_Test_2D)):
    if i%4 == 0:
        count = count+1
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400, label=str(int(i/4)))
    else:
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400)
        
#ax.legend(bbox_to_anchor=(0.999, 1.00))
ax.legend(loc="best", ncol=4, fontsize=fz*0.75)
#ax.legend(loc="lower left", mode = "expand", ncol=24, fontsize=fz/2)
ax.set_title("Latent Space from Amazon Average Diurnal Cycle Colored by LST Hour", fontsize=fz*2)

#plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Amazon.pdf")
print(" ")

In [ ]:
colors = cm.hsv(np.linspace(0, 1, int(len(Diurnal_Amazon_W_Test_2D)/4)))
bc_labels = ["0","1","2","3", "4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
fig, ax = plt.subplots(figsize=(30, 10))
ax.scatter(x=z_test_tsne[:, 0], y=z_test_tsne[:, 1], c="black", s=0.2)
count = -1
for i in range(len(Diurnal_Atlantic_W_Test_2D)):
    if i%4 == 0:
        count = count+1
        cb = ax.scatter(x=Diurnal_Atlantic_W_Test_2D[i, 0], y=Diurnal_Atlantic_W_Test_2D[i, 1], c=colors[count], s=400, label=str(int(i/4)))
    else:
        cb = ax.scatter(x=Diurnal_Atlantic_W_Test_2D[i, 0], y=Diurnal_Atlantic_W_Test_2D[i, 1], c=colors[count], s=400)
        
#ax.legend(bbox_to_anchor=(0.999, 1.00))
ax.legend(loc="best", ncol=4, fontsize=fz*0.75)
#ax.legend(loc="lower left", mode = "expand", ncol=24, fontsize=fz/2)
ax.set_title("Latent Space from Amazon Average Diurnal Cycle Colored by LST Hour", fontsize=fz*2)

#plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Amazon.pdf")
print(" ")

In [ ]:
Test_Images_Reorganized = np.zeros(shape=(int(len(All_Test_Images)/(22*144)),30,128,22,144))
Test_Latents_Reorganized = np.zeros(shape=(int(len(z_test_tsne_track)/(22*144)),2,22,144))
Test_Images_LST_Labels = np.zeros(shape=(96,30,128,22,144))
Test_Latents_LST_Labels = np.zeros(shape=(96,2,22,144))
Test_Images_LST_Labels_Sorted = np.zeros(shape=(96,30,128,22,144))
Test_Latents_LST_Labels_Sorted = np.zeros(shape=(96,2,22,144))
filler = np.arange(0,96,1)
filler_a = filler[:64]
filler_b = filler[64:]
filler = np.concatenate((filler_b, filler_a))

In [ ]:
count = 0
for i in range(len(Test_Images_Reorganized)):
    for j in range(len(Test_Images_Reorganized[0][0][0])): 
        for k in range(len(Test_Images_Reorganized[0][0][0][0])): 
            Test_Images_Reorganized[i,:,:,j,k] = All_Test_Images[count,:,:]
            Test_Latents_Reorganized[i,:,j,k] = z_test_tsne_track[count,:]
            count = count+1

In [ ]:
count = 0
for i in range(len(Test_Images_LST_Labels[0])):
    for j in range(len(Test_Images_LST_Labels[0][0])): 
        for k in range(len(Test_Images_LST_Labels[0][0][0])): 
            for l in range(len(Test_Images_LST_Labels[0][0][0][0])):
                Test_Images_LST_Labels[:,i,j,k,l] = filler
                count = count+1
            
count = 0
for i in range(len(Test_Latents_LST_Labels[0])):
    for j in range(len(Test_Latents_LST_Labels[0][0])): 
        for k in range(len(Test_Latents_LST_Labels[0][0][0])): 
            Test_Latents_LST_Labels[:,i,j,k] = filler
            count = count+1

In [ ]:
for i in range(len(Test_Images_LST_Labels[0][0][0][0])):
    splitter = int(round((lons[i]/360.)*96.))
    part_a = Test_Images_LST_Labels[:-splitter,:,:,:,i]
    part_b = Test_Images_LST_Labels[-splitter:,:,:,:,i]
    combined = np.concatenate((part_b, part_a), axis=0)
    Test_Images_LST_Labels_Sorted[:,:,:,:,i] = combined
    
    part_a = Test_Latents_LST_Labels[:-splitter,:,:,i]
    part_b = Test_Latents_LST_Labels[-splitter:,:,:,i]
    combined = np.concatenate((part_b, part_a), axis=0)
    Test_Latents_LST_Labels_Sorted[:,:,:,i] = combined

In [ ]:
for i in range(len(Test_Images_LST_Labels[0][0][0][0])):
    splitter = int(round((lons[i]/360.)*96.))
    part_a = Test_Images_LST_Labels[splitter:,:,:,:,i]
    part_b = Test_Images_LST_Labels[:splitter,:,:,:,i]
    combined = np.concatenate((part_a, part_b), axis=0)
    Test_Images_LST_Labels_Sorted[:,:,:,:,i] = combined
    
    part_a = Test_Latents_LST_Labels[splitter:,:,:,i]
    part_b = Test_Latents_LST_Labels[:splitter,:,:,i]
    combined = np.concatenate((part_a, part_b), axis=0)
    Test_Latents_LST_Labels_Sorted[:,:,:,i] = combined

In [ ]:
Final_LST_Image_Labels = Test_Images_LST_Labels_Sorted[:16,:,:,:,:]
Final_LST_Latent_Labels = Test_Latents_LST_Labels_Sorted[:16,:,:,:]

In [ ]:
for i in range(len(Final_LST_Latent_Labels[0][0][0])):
    print(Final_LST_Latent_Labels[:,0,0,i], lons[i])

In [ ]:
Truly_Final_LST_Image_Labels = np.empty(All_Test_Images.shape)
Truly_Final_LST_Latent_Labels = np.empty(z_test_tsne_track.shape)

count = 0
for i in range(len(Final_LST_Image_Labels)):
    for j in range(len(Final_LST_Image_Labels[0][0][0])):
        for k in range(len(Final_LST_Image_Labels[0][0][0][0])):
            Truly_Final_LST_Image_Labels[count,:,:] = Final_LST_Image_Labels[i,:,:,j,k]
            count = count+1
            
count = 0
for i in range(len(Final_LST_Latent_Labels)):
    for j in range(len(Final_LST_Latent_Labels[0][0])):
        for k in range(len(Final_LST_Latent_Labels[0][0][0])):
            Truly_Final_LST_Latent_Labels[count,:] = Final_LST_Latent_Labels[i,:,j,k]
            count = count+1

In [ ]:
Useful_LTS_Labels = Truly_Final_LST_Latent_Labels[:,0]

In [ ]:
colors = cm.hsv(np.linspace(0, 1, int(len(Diurnal_Amazon_W_Test_2D)/4)))
bc_labels = ["0","1","2","3", "4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
fig, ax = plt.subplots(figsize=(30, 10))
obs = []
for i in range(10000):
#for i in range(len(z_test_tsne_track)):
    if i %1000 == 0:
        print(i)
    if int(Useful_LTS_Labels[i]/4) not in obs:
        obs.append(int(Useful_LTS_Labels[i]/4))
        ax.scatter(x=z_test_tsne_track[i, 0], y=z_test_tsne_track[i, 1], c=colors[int(Useful_LTS_Labels[i]/4)], s=10.0, label = str(int(Useful_LTS_Labels[i]/4)))
    else:
        ax.scatter(x=z_test_tsne_track[i, 0], y=z_test_tsne_track[i, 1], c=colors[int(Useful_LTS_Labels[i]/4)], s=10.0)
        
       

lgnd = ax.legend(loc="best", ncol=4, fontsize=fz*0.75)
for handle in lgnd.legendHandles:
    handle.set_sizes([200.0])
    
    
count = -1
for i in range(len(Diurnal_Amazon_W_Test_2D)):
    if i%4 == 0:
        count = count+1
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400, label=str(int(i/4)))
    else:
        cb = ax.scatter(x=Diurnal_Amazon_W_Test_2D[i, 0], y=Diurnal_Amazon_W_Test_2D[i, 1], c=colors[count], s=400)
 
#ax.legend(loc="lower left", mode = "expand", ncol=24, fontsize=fz/2)
ax.set_title("Latent Space from Amazon Average Diurnal Cycle Colored by LST Hour", fontsize=fz*2)

#plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Amazon.pdf")
print(" ")